In [1]:
import numpy as np
import matplotlib.pyplot as plt# display plots in this notebook
import copy
import os.path as osp
import glob
import shutil  
import pandas as pd
import sys, caffe, os
%matplotlib inline

plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'  # use grayscale output rather than the color one for display

# this file should be run from {caffe_root}/examples (otherwise change this line)
caffe_root = '/home/bit/temp2/caffe/'
sys.path.insert(0, caffe_root + 'python')

caffe.set_mode_gpu()

work_root    = caffe_root + 'examples/shipHead8/'
model_def     = work_root + 'deploy.prototxt'
model_weights = work_root + 'shiphead8_iter_60000.caffemodel'

net = caffe.Net(model_def, model_weights, caffe.TEST)
mu = np.array([1.0, 1.0, 1.0])
#print 'mean-subtracted values:', zip('BGR', mu)

In [2]:
# create transformer for the input called 'data'
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})

transformer.set_transpose('data', (2,0,1))  # move image channels to outermost dimension
transformer.set_raw_scale('data', 2)      # rescale from [0, 1] to [0, 2]
transformer.set_mean('data', mu)            # subtract the dataset-mean value in each channel
transformer.set_channel_swap('data', (2,1,0))  # swap channels from RGB to BGR

# set the size of the input (we can skip this if we're happy
#  with the default; we can also change it later, e.g., for different batch sizes)
net.blobs['data'].reshape(1,        # batch size
                          3,         # 3-channel (BGR) images
                          40, 40)  # image size

In [3]:
shutil.copyfile( work_root + 'shipheadtrain/shiphead.txt', work_root + 'shipheadnew.txt')

data_list = pd.read_table( work_root + 'shipheadtrain/shiphead.txt', header=None,sep='[ |.]',engine='python')
#print data_list
#print data_list.iat[0,0]
#print len(data_list)

In [4]:
 with open(work_root + 'shipheadnew.txt',"a") as f:
            new_context = '\n'
            f.write( new_context )


#len(data_list)
for i in xrange( len(data_list) ):
    if i%10000 == 1:
        print i
    ind = data_list.iat[i,0]
    image_path = osp.join(caffe_root+'examples/shipHead8/shipheadtrain', str(np.int_(ind))+'.jpg')

    image = caffe.io.load_image(image_path)
    transformed_image = transformer.preprocess('data', image)

    # copy the image data into the memory allocated for the net
    net.blobs['data'].data[...] = transformed_image

    output = net.forward()
    output_prob = output['prob'][0]  # the output probability vector for the first image in the batch

#    print output_prob
#    print 'predicted class is:', output_prob.argmax()
#    if image.ndim == 3:
#        im1 = image[:,:,0]            
#    plt.imshow(im1)

    if data_list.iat[i,2] != output_prob.argmax():
#        with open(work_root + 'shipheadtrain/shipheadnew.txt',"a") as f:
        with open(work_root + 'shipheadnew.txt',"a") as f:
            new_context = str(np.int_(ind))+'.jpg'+ ' ' +str(data_list.iat[i,2]) + '\n'
            f.write( new_context )

1
10001
20001
30001
40001
50001
60001
70001
80001
90001
100001
110001
120001
130001
140001
150001
160001
170001
180001
190001
200001
210001
220001
230001
240001
250001
260001
270001
280001
290001
300001
310001
320001
330001
340001
350001
360001
370001
380001
390001
400001
410001
420001
430001
440001
450001
460001
470001
480001
490001
500001
510001
520001
530001
540001
550001
560001
570001
580001
590001
600001
610001
620001
630001
640001
650001
660001
670001
680001
690001
700001
710001
720001
730001
740001
750001
760001
